# Semana 3 en JupyterLab

In [1]:
# Paquetes necesarios para JupyterLab
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate gradio datasets

In [2]:
# En mi versión de linux se requiere actualizar el paquete tokenizador
!pip install -U -q bitsandbytes

In [3]:
# Importads
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr
from datasets import load_dataset

In [4]:
# Requiere digitar token de HuggingFace
notebook_login()

In [5]:
# instruct models

# Para efectos prácticos y por la memoria de mi GPU (4GB) utilizo un modelo Llama3.2 1B
LLAMA = "meta-llama/Llama-3.2-1B-Instruct"

# Load dateset de HuggingFace sobre productos en español, solo se toma los 10 primeros
ds = load_dataset("Globaly/655k-productos-clasificados", split='train[0:10]')

# Variable global
listado = ''
# print(ds.num_rows)
# print(ds.info)

# Listado con los 10 primeros productos
for i in range(ds.num_rows):
  listado += ds[i]['input'] + ', '
  # print(ds[i]['input'])

# print(listado)

In [6]:
# Configuración de cuantificación: esto nos permite cargar el modelo en la GPU y usar menos memoria.

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [7]:
# Prompt del modelo pasando el listado
system_message = "Eres un asistente que realiza breves comentarios y caracteristicas positivas sobre productos en español, en formato Markdown."
user_prompt = f"Genera comentario de los siguientes productos. Escriba las actas en Markdown en español.\n{listado}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [8]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
modelo = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="auto",
    quantization_config=quant_config
)

In [9]:
def generate_text(prompt):
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)
    outputs = modelo.generate(inputs, max_new_tokens=2000, streamer=streamer)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

In [12]:
# El botón Summit realiza el llamado a la función, el promp no cumple función.
demo = gr.Interface(fn=generate_text, inputs="text", outputs="text")
demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Feb 2025

Eres un asistente que realiza breves comentarios y caracteristicas positivas sobre productos en español, en formato Markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Genera comentario de los siguientes productos. Escriba las actas en Markdown en español.
Ajustable,<|eot_id|><|start_header_id|>assistant<|end_header_id|>río Compacta Multicapas para Excursión y Picnic, Nevera Rígida de 50 Litros Resistente a Impactos para Acampada, Mochila Enfriadora Impermeable para Bebidas y Alimentos, Bolsa de Almuerzo Aislante Térmico para Días de Campo, Porta Alimentos Térmico de 30 Litros para Picnic al Aire Libre, Nevera de Camping XL de 60 litros de Capacidad, Bolsa de Picnic con Aislamiento Térmico y Asa Acolchada, Nevera Portátil Ligera de 40L para Alimentos y Bebidas, Porta Alimentos de Gran Capacidad para Acampada con Correa 

Aquí te dejo los comentarios en fo